In [1]:
!pip install pyrosettacolabsetup
!pip install MOSEK
!pip install CLARABEL

import pyrosettacolabsetup; pyrosettacolabsetup.install_pyrosetta()

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 15.1/15.1 MB 1.8 MB/s eta 0:00:00
Mounted at /content/google_drive

Note that USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE.
See https://github.com/RosettaCommons/rosetta/blob/main/LICENSE.md or email license@uw.edu for details.

Looking for compatible PyRosetta wheel file at google-drive/PyRosetta/colab.bin//wheels...
Found compatible wheel: /content/google_drive/MyDrive/PyRosetta/colab.bin/wheels//content/google_drive/MyDrive/PyRosetta/colab.bin/wheels/pyrosetta-2024.10+release.2c36cbc710-cp310-cp310-linux_x86_64.whl




In [2]:
from pyrosetta import *
from pyrosetta.toolbox import pose_from_rcsb
from pyrosetta.toolbox.extract_coords_pose import pose_coords_as_rows
from pyrosetta.teaching import *

pyrosetta.init()

┌──────────────────────────────────────────────────────────────────────────────┐
│                                 PyRosetta-4                                  │
│              Created in JHU by Sergey Lyskov and PyRosetta Team              │
│              (C) Copyright Rosetta Commons Member Institutions               │
│                                                                              │
│ NOTE: USE OF PyRosetta FOR COMMERCIAL PURPOSES REQUIRE PURCHASE OF A LICENSE │
│              See LICENSE.md or email license@uw.edu for details              │
└──────────────────────────────────────────────────────────────────────────────┘
PyRosetta-4 2024 [Rosetta PyRosetta4.MinSizeRel.python310.ubuntu 2024.10+release.2c36cbc7108d85646ca5b8ddc89c29ac1ccde88e 2024-03-01T16:53:36] retrieved from: http://www.pyrosetta.org
core.init: Checking for fconfig files in pwd and ./rosetta/flags
core.init: Rosetta version: PyRosetta4.MinSizeRel.python310.ubuntu r372 2024.10+release.2c36cbc710 2c36

In [3]:
import sys

def num_chis(pose, res_num):
    counter = 0

    while True:
        try:
            u = pose.chi(counter + 1, res_num)
            counter += 1
        except Exception:
            break

    return counter

def num_residues(pose):
    counter = 0

    while True:
        try:
            u = pose.psi(counter + 1)
            counter += 1
        except Exception:
            break

    return counter

def unfold(pose):
    new_pose = Pose()
    new_pose.detached_copy(pose)
    # for i in range(1, num_residues(new_pose) + 1):
    for i in range(1, new_pose.total_residue() + 1):
        new_pose.set_phi(i, 180)
        new_pose.set_psi(i, -180)
        for j in range(1, num_chis(new_pose, i) + 1):
            new_pose.set_chi(j, i, 0)
    return new_pose

In [4]:
import numpy as np
import numpy.linalg as la

# def get_coordinates(pose):
#     coordinates = []
#     for i in range(1, num_residues(pose) + 1):
#         j = 1
#         while True:
#             if pose.residue(i).xyz(j)[0] * 1000 - np.trunc(pose.residue(i).xyz(j)[0] * 1000) != 0:
#                 break
#             coordinates.append(np.array([pose.residue(i).xyz(j)[0], pose.residue(i).xyz(j)[1], pose.residue(i).xyz(j)[2]]))
#             j += 1
#     return np.array(coordinates)

def is_valid(pose, tolerance):
    coordinates = pose_coords_as_rows(pose)
    for i in range(coordinates.shape[0]):
        for j in range(i + 1, coordinates.shape[0]):
            if la.norm(coordinates[i] - coordinates[j]) < tolerance:
                return False
    return True

def num_angles(pose):
    count = 0
    for i in range(1, num_residues(pose) + 1):
        count += num_chis(pose, i)
    count += 2 * num_residues(pose)
    return count

def generate_noise(d, epsilon):
    inv_d = 1.0 / d
    gauss = np.random.normal(size=d)
    length = la.norm(gauss, 1)
    r = np.random.rand() ** inv_d
    return epsilon * np.multiply(gauss, r / length)

def create_sample(pose, epsilon):
    new_pose = Pose()
    new_pose.detached_copy(pose)
    noise = generate_noise(num_angles(pose), epsilon)
    idx = 0
    for i in range(1, new_pose.total_residue() + 1):
        new_pose.set_phi(i, pose.phi(i) + noise[idx])
        idx += 1
        new_pose.set_psi(i, pose.psi(i) + noise[idx])
        idx += 1
        for j in range(1, num_chis(new_pose, i) + 1):
            new_pose.set_chi(j, i, pose.chi(j, i) + noise[idx])
            idx += 1
    return [noise, new_pose]

In [5]:
def slice_pose(pose, start, end):
    new_pose = Pose()
    ind = pyrosetta.rosetta.utility.vector1_unsigned_long()
    for i in range(start, end + 1):
        ind.append(i)
    pyrosetta.rosetta.core.pose.pdbslice(new_pose, pose, ind)
    return new_pose

In [6]:
def direct_product(set_1, set_2):
    out = []
    for el_1 in set_1:
        for el_2 in set_2:
            if isinstance(set_1[0], list) and isinstance(set_2[0], list):
                out.append(el_1.copy() + el_2.copy())
            elif isinstance(set_1[0], list) and not isinstance(set_2[0], list):
                out.append(el_1.copy() + [el_2])
            elif not isinstance(set_1[0], list) and isinstance(set_2[0], list):
                out.append([el_1] + el_2.copy())
            else:
                out.append([el_1] + [el_2])
    return out

def generate_grid(bound, gap, dimension):
    grid1D = np.arange(-bound, bound + gap, gap)
    out = direct_product(grid1D, grid1D)
    for i in range(dimension - 2):
        out = direct_product(out, grid1D)
    return out

def extract_net(grid, epsilon):
    out = []
    for el in grid:
        if np.sum(np.abs(el)) == epsilon:
            out.append(el)
    return out

def perturb_pose(pose, noise):
    new_pose = Pose()
    new_pose.detached_copy(pose)
    idx = 2 * new_pose.total_residue()

    for i in range(1, new_pose.total_residue() + 1):
        new_pose.set_phi(i, pose.phi(i) + noise[2 * i - 2])
        new_pose.set_psi(i, pose.psi(i) + noise[2 * i - 1])

        for j in range(1, num_chis(new_pose, i) + 1):
            new_pose.set_chi(j, i, new_pose.chi(j, i) + noise[idx])
            idx += 1
    return new_pose

def generate_landscape(pose, bound, gap):
    sfxn = get_score_function(True)
    landscape = []
    grid = generate_grid(bound, gap, num_angles(pose))
    for el in grid:
        perturbed_pose = perturb_pose(pose, el)
        landscape.append(sfxn(perturbed_pose))
    return landscape

def random_walk(pose, bound, gap, tolerance, limit):
    path = []
    N = num_angles(pose)
    path.append(np.zeros(N))

    while True:
        if la.norm(path[-1], 1) == bound:
            return path
        else:
            for i in range(limit):
                idx = np.random.randint(N)
                bin = np.random.randint(2)
                if bin == 0:
                    noise = path[-1].copy()
                    noise[idx] = noise[idx] - gap
                    if is_valid(perturb_pose(pose, noise), tolerance):
                        path.append(noise)
                        break
                    else:
                        print("invalid " + str(la.norm(noise, 1)))
                        continue
                else:
                    noise = path[-1].copy()
                    noise[idx] = noise[idx] + gap
                    if is_valid(perturb_pose(pose, noise), tolerance):
                        path.append(noise)
                        break
                    else:
                        print("invalid " + str(la.norm(noise, 1)))
                        continue


In [7]:
import matplotlib.pyplot as plt
import multiprocessing as mp

def plot_path(pose, path):
    x = []
    y = []
    sizes = [5] * len(path)
    sfxn = get_score_function(True)

    for noise in path:
        x.append(la.norm(noise, 1))
        y.append(sfxn(perturb_pose(pose, noise)))

    plt.scatter(x, y, s = sizes)
    plt.show()

def plot_landscape(pose, bound, gap, tolerance, limit, plot_invalids = False):
    level = 0
    N = num_angles(pose)
    x = []
    y = []
    z = []
    sfxn = get_score_function(True)

    while level != bound:
        energies = []

        for i in range(limit):
            u = np.random.uniform(0.0, 1.0, N)
            noise = level * (u / la.norm(u, 1))
            if is_valid(perturb_pose(pose, noise), tolerance):
                energies.append(sfxn(perturb_pose(pose, noise)))

        energies = np.array(energies)
        z.append(len(energies) / limit)
        y.append(np.mean(energies))
        x.append(level)
        level += gap

    if plot_invalids:
        plt.plot(x, z)
        plt.show()
    else:
        plt.plot(x, y)
        plt.show()


In [8]:
def rmsd(pose_1, pose_2):
    coords_1 = pose_coords_as_rows(pose_1)
    coords_2 = pose_coords_as_rows(pose_2)

    sum = 0
    for i in range(coords_1.shape[0]):
        sum += la.norm(coords_1[i] - coords_2[i]) ** 2

    return np.sqrt(sum / coords_1.shape[0])

def estimate_lipschitz(num_samples, pose, boundary, gap, tolerance):
    sfxn = get_score_function(True)
    N = num_angles(pose)
    max_pose = pose
    max_slope = 0

    for i in range(num_samples):
        noise = generate_noise(N, boundary)
        new_pose = perturb_pose(pose, noise)

        idx = np.random.randint(N)
        bin = np.random.randint(2)
        perturbation = np.zeros(N)
        perturbation[idx] += (-1) ** bin * gap

        new_pose_neighbor = perturb_pose(new_pose, perturbation)

        if is_valid(new_pose_neighbor, tolerance) and is_valid(new_pose, tolerance):
            slope = np.abs(sfxn(new_pose_neighbor) - sfxn(new_pose)) / gap
            if slope > max_slope:
                max_slope = slope
                max_poses = [new_pose, new_pose_neighbor]

    return max_slope, max_poses

In [9]:
def iterator(n, d, gap, bound):
    bound = float(bound)
    length = 2 * (bound / gap) + 1
    out = np.array([-bound] * d)
    rem = n

    for i in range(d - 1, 0, -1):
        quot = rem // (length ** i)
        rem = rem % (length ** i)
        out[d - 1 - i] += quot * gap

    out[d - 1] += rem * gap
    return out

In [10]:
def bond_lengths(pose):
    lengths = []
    min = 100
    for i in range(1, pose.total_residue() + 1):
        for l in range(1, pose.total_residue() + 1):
            for j in range(1, pose.residue(i).natoms() + 1):
                for k in range(1, pose.residue(l).natoms() + 1):
                    dist = la.norm(pose.residue(i).xyz(j) - pose.residue(l).xyz(k))
                    if pose.conformation().is_bonded(AtomID(j, i), AtomID(k, l)):
                        lengths.append(dist)
                    else:
                        if dist < min and (i, j) != (l, k):
                            min = dist

    return np.array(list(set(lengths))), min

In [11]:
import networkx as nx
import random

def generate_fold_graph(pose):
    G = nx.Graph()

    for i in range(1, pose.total_residue() + 1):
        for j in range(1, pose.residue(i).natoms() + 1):
            G.add_node((i, j), coords = pose.residue(i).xyz(j))

    for i in range(1, pose.total_residue() + 1):
        for l in range(1, pose.total_residue() + 1):
            for j in range(1, pose.residue(i).natoms() + 1):
                for k in range(1, pose.residue(l).natoms() + 1):
                    if pose.conformation().is_bonded(AtomID(j, i), AtomID(k, l)):
                        G.add_edge((i, j), (l, k), dist = la.norm(pose.residue(i).xyz(j) - pose.residue(l).xyz(k)))

    return G

def hierarchy_pos(G, root, levels=None, width=1., height=1.):
    '''If there is a cycle that is reachable from root, then this will see infinite recursion.
       G: the graph
       root: the root node
       levels: a dictionary
               key: level number (starting from 0)
               value: number of nodes in this level
       width: horizontal space allocated for drawing
       height: vertical space allocated for drawing'''
    TOTAL = "total"
    CURRENT = "current"
    def make_levels(levels, node=root, currentLevel=0, parent=None):
        """Compute the number of nodes for each level
        """
        if not currentLevel in levels:
            levels[currentLevel] = {TOTAL : 0, CURRENT : 0}
        levels[currentLevel][TOTAL] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                levels =  make_levels(levels, neighbor, currentLevel + 1, node)
        return levels

    def make_pos(pos, node=root, currentLevel=0, parent=None, vert_loc=0):
        dx = 1/levels[currentLevel][TOTAL]
        left = dx/2
        pos[node] = ((left + dx*levels[currentLevel][CURRENT])*width, vert_loc)
        levels[currentLevel][CURRENT] += 1
        neighbors = G.neighbors(node)
        for neighbor in neighbors:
            if not neighbor == parent:
                pos = make_pos(pos, neighbor, currentLevel + 1, node, vert_loc-vert_gap)
        return pos
    if levels is None:
        levels = make_levels({})
    else:
        levels = {l:{TOTAL: levels[l], CURRENT:0} for l in levels}
    vert_gap = height / (max([l for l in levels])+1)
    return make_pos({})

def extract_side_chain(i, G):
    G_ = G.copy()
    G_.remove_edge((i, 2), (i, 5))
    return G.subgraph(list(nx.dfs_preorder_nodes(G_, source=(i, 5))))

def slice_pose(pose, start, end):
    new_pose = Pose()
    ind = pyrosetta.rosetta.utility.vector1_unsigned_long()
    for i in range(start, end + 1):
        ind.append(i)
    pyrosetta.rosetta.core.pose.pdbslice(new_pose, pose, ind)
    return new_pose

In [12]:
lockr = pose_from_pdb('7jh5.pdb')
lockr_helix = slice_pose(lockr, 1, 48)
# lockr_subhelix = slice_pose(lockr, 274, 278)
# dump_pdb(lockr_subhelix, 'lockr_subhelix.pdb')

core.chemical.GlobalResidueTypeSet: Finished initializing fa_standard residue type set.  Created 985 residue types
core.chemical.GlobalResidueTypeSet: Total time to initialize 1.75617 seconds.
core.import_pose.import_pose: File '7jh5.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OG  on residue SER:NtermProteinFull 1
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OG  on residue SER 3
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CG  on residue GLU 4
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue GLU 4
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE1 on residue GLU 4
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OE2 on residue GLU 4
core.conformation.Conformation: [ WARNING ] missing heavyatom:  CD  on residue ARG 7
core.conformation.Conformation: [ WARNING ] missing heavyatom:  NE  on residue ARG 7
core.conformation.Con

In [13]:
!pip install cvxpy
import cvxpy as cp

def D(pose):
    coords = pose_coords_as_rows(pose)
    N = len(coords)
    D = np.zeros((N, N))

    for i in range(N):
        for j in range(N):
            D[i, j] = la.norm(coords[i] - coords[j]) ** 2
    return D

def K_of(D):
    N = D.shape[0]
    I = np.identity(N)
    J = np.ones((N, N))
    C = I - J / N

    return -0.5 * C @ D @ C

def d(i, j, K):
    return K[i, i] + K[j, j] - 2 * K[i, j]

def stringify(vec):
    out = ""
    for i in range(len(vec)):
        out += " " + str(vec[i]) + " "
    return out

def coords_to_AtomID_dict(pose):
    dic = {}
    for i in range(1, pose.total_residue() + 1):
        for j in range(1, pose.residue(i).natoms() + 1):
            dic[stringify(pose.residue(i).xyz(j))] = AtomID(j, i)
    return dic

def AtomID_idx_to_coords_idx_dict(pose):
    dic = {}
    coords = pose_coords_as_rows(pose)

    for i in range(1, pose.total_residue() + 1):
        for j in range(1, pose.residue(i).natoms() + 1):
            for k in range(coords.shape[0]):
                if la.norm(coords[k] - pose.residue(i).xyz(j)) == 0:
                    dic[(i, j)] = k

    return dic

def coords_idx_to_AtomID_idx_dict(pose):
    dic = {}
    coords = pose_coords_as_rows(pose)

    for i in range(1, pose.total_residue() + 1):
        for j in range(1, pose.residue(i).natoms() + 1):
            for k in range(coords.shape[0]):
                if la.norm(coords[k] - pose.residue(i).xyz(j)) == 0:
                    dic[k] = (i,j)

    return dic

def potential_K(K, base_pose):
    dic_1 = coords_idx_to_AtomID_idx_dict(base_pose)
    dic_2 = coords_to_AtomID_dict(base_pose)
    coords = pose_coords_as_rows(base_pose)
    sfxn = ScoreFunction()
    # sfxn.set_weight(fa_atr, 1.0)
    sfxn.set_weight(fa_rep, 1.0)
    out = 0

    for i in range(coords.shape[0]):
        for j in range(i + 1, coords.shape[0]):
            id_i = dic_2[stringify(coords[i])]
            id_j = dic_2[stringify(coords[j])]

            if not base_pose.conformation().is_bonded(id_i, id_j):
                k, l = dic_1[i]
                m, n = dic_1[j]
                out += pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies(base_pose.residue(k), l, base_pose.residue(m), n, sfxn)[0]
    return out

In [14]:
def aa_energy(pose, m, j, n, k, dist, sfxn):
    pose_copy = Pose()
    pose_copy.detached_copy(pose)
    res1 = pose_copy.residue(m)
    res2 = pose_copy.residue(n)
    pos_1 = np.array(res1.atom(j).xyz())
    pos_2 = np.array(res2.atom(k).xyz())
    dir = pos_1 + dist * (pos_2 - pos_1) / la.norm(pos_2 - pos_1)
    new_pos = pyrosetta.rosetta.numeric.xyzVector_double_t()
    new_pos.assign(dir[0], dir[1], dir[2])
    res2.atom(k).xyz(new_pos)

    score_manager = rosetta.core.scoring.ScoringManager.get_instance()
    etable_ptr = score_manager.etable(sfxn.energy_method_options().etable_type())
    etable = etable_ptr.lock()
    etable_energy = rosetta.core.scoring.etable.AnalyticEtableEnergy(
        etable, sfxn.energy_method_options()
    )

    # Construct coulomb class for calculating fa_elec energies
    coulomb = rosetta.core.scoring.etable.coulomb.Coulomb(sfxn.energy_method_options())

    # Construct AtomPairEnergy container to hold computed energies.
    ape = rosetta.core.scoring.etable.AtomPairEnergy()

    # Set all energies in the AtomPairEnergy to zero prior to calculation.
    ape.attractive, ape.bead_bead_interaction, ape.repulsive, ape.solvation = [0.] * 4

    # Calculate the distance squared and set it in the AtomPairEnergy.
    ape.distance_squared = res1.xyz(j).distance_squared(
        res2.xyz(k)
    )

    # Evaluate energies from pre-calculated etable, using a weight of 1.0
    # in order to match the raw energies from eval_ci_2b.
    atom1 = res1.atom(j)
    atom2 = res2.atom(k)
    etable_energy.atom_pair_energy(atom1, atom2, 1.0, ape)

    # Calculate atom-atom scores.
    lj_atr = ape.attractive
    lj_rep = ape.repulsive
    solv = ape.solvation
    fa_elec = coulomb.eval_atom_atom_fa_elecE(
        res1.xyz(j),
        res1.atomic_charge(j),
        res2.xyz(k),
        res2.atomic_charge(k),
    )

    return lj_atr, lj_rep, solv, fa_elec

def w(m, j, n, k, spl):
    dist = spl[(m, j)][(n, k)]
    if dist <= 3:
        return 0
    elif dist == 4:
        return 0.2
    else:
        return 1

def rr_energy(pose, sfxn, m, n):
    if m == n:
        return 0

    out = 0
    G = generate_fold_graph(pose)
    spl = dict(nx.all_pairs_shortest_path_length(G))

    for j in range(1, pose.residue(m).natoms() + 1):
        for k in range(1, pose.residue(n).natoms() + 1):
            if np.abs(m - n) == 1:
                out += w(m, j, n, k, spl) * pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies(pose.residue(m), j, pose.residue(n), k, sfxn)[0]
            else:
                out += pyrosetta.toolbox.atom_pair_energy.etable_atom_pair_energies(pose.residue(m), j, pose.residue(n), k, sfxn)[0]
    return out

def approximate_sigma(pose, m, j, n, k, start, end, sfxn):
    if np.isclose(start, end, rtol = 1e-20):
        return start

    if aa_energy(pose, m, j, n, k, (start + end) / 2, sfxn)[0] == aa_energy(pose, m, j, n, k, 0, sfxn)[0]:
        return approximate_sigma(pose, m, j, n, k, (start + end) / 2, end, sfxn)
    else:
        return approximate_sigma(pose, m, j, n, k, start, (start + end) / 2, sfxn)

def constant_dictionary(pose, sfxn):
    dic = {}
    for m in range(1, pose.total_residue() + 1):
        for n in range(1, pose.total_residue() + 1):
            for j in range(1, pose.residue(m).natoms() + 1):
                for k in range(1, pose.residue(n).natoms() + 1):
                    dic[((m, j), (n, k))] = (-aa_energy(pose, m, j, n, k, 0, sfxn)[0], approximate_sigma(pose, m, j, n, k, 0, 6, sfxn))
    return dic

def derive_f_poly(pose, m, j, n, k, sfxn):
    b = np.array([aa_energy(pose, m, j, n, k, 4.5, sfxn)[0],
                  aa_energy(pose, m, j, n, k, 5, sfxn)[0],
                  aa_energy(pose, m, j, n, k, 5.5, sfxn)[0],
                  aa_energy(pose, m, j, n, k, 6, sfxn)[0]]).T

    A = np.array([[1, 4.5, 4.5 ** 2, 4.5 ** 3],
                  [1, 5, 5 ** 2, 5 ** 3],
                  [1, 5.5, 5.5 ** 2, 5.5 ** 3],
                  [1, 6, 6 ** 2, 6 ** 3]])

    return la.inv(A) @ b

def E_atr(K, base_pose, const_dict, AtomID_to_coords_dict):
    G = generate_fold_graph(base_pose)
    spl = dict(nx.all_pairs_shortest_path_length(G))
    sum = 0

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    epsilon, sigma = const_dict[((m, j), (n, k))]
                    dist = np.sqrt(d(AtomID_to_coords_dict[(m, j)], AtomID_to_coords_dict[(n, k)], K))
                    term = 0

                    if dist <= sigma:
                        term = -epsilon
                    elif dist <= 4.5:
                        term = epsilon * ((sigma / dist) ** 12 - 2 * (sigma / dist) ** 6)
                    elif dist <= 6:
                        term = np.dot(np.array([1, dist, dist ** 2, dist ** 3]), derive_f_poly(lockr_subhelix, m, j, n, k, sfxn))
                    else:
                        term = 0

                    if np.abs(m - n) == 1:
                        sum += w(m, j, n, k, spl) * term
                    else:
                        sum += term

    return sum

def generate_linear_E_atr_dict(base_pose, const_dict, AtomID_to_coords_dict):
    G = generate_fold_graph(base_pose)
    spl = dict(nx.all_pairs_shortest_path_length(G))
    dic = {}
    D_base = D(base_pose)

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    epsilon, sigma = const_dict[((m, j), (n, k))]
                    base_dist = np.sqrt(D_base[AtomID_to_coords_dict[(m, j)]][AtomID_to_coords_dict[(n, k)]])
                    term = 0

                    if base_dist <= sigma:
                        dic[((m, j), (n, k))] = -epsilon
                    elif base_dist <= 4.5:
                        slope = 6 * epsilon * (sigma ** 6 / base_dist ** 8 - sigma ** 12 / base_dist ** 14)
                        intercept = epsilon * ((sigma / base_dist) ** 12 - 2 * (sigma / base_dist) ** 6)
                        dic[((m, j), (n, k))] = (slope, intercept, base_dist)
                    elif base_dist <= 6:
                        coeffs = derive_f_poly(lockr_subhelix, m, j, n, k, sfxn)
                        slope = coeffs[1] / (2 * base_dist) + coeffs[2] + 3 * coeffs[3] * base_dist / 2
                        intercept = np.dot(np.array([1, base_dist, base_dist ** 2, base_dist ** 3]), coeffs)
                        dic[((m, j), (n, k))] = (slope, intercept, base_dist)
                    else:
                        dic[((m, j), (n, k))] = 0

    return dic, spl

def linear_E_atr(K, base_pose, linear_E_atr_dict, spl, AtomID_to_coords_dict):
    sum = 0

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    constants = linear_E_atr_dict[((m, j), (n, k))]
                    dist_squared = d(AtomID_to_coords_dict[(m, j)], AtomID_to_coords_dict[(n, k)], K)
                    term = 0

                    if type(constants) == tuple:
                        slope, intercept, base_dist = constants
                        term = intercept + slope * (dist_squared - base_dist ** 2)
                    else:
                        term = constants

                    if np.abs(m - n) == 1:
                        sum += w(m, j, n, k, spl) * term
                    else:
                        sum += term

    return sum

def objective(K, base_pose, c, sfxn, const_dict, AtomID_to_coords_dict):
    return E_atr(K, base_pose, const_dict,AtomID_to_coords_dict) - sfxn(base_pose)
    #  - c * la.norm(D - D_base, 1)

def closest_points(k, point, point_list):
    closest_list = []

    for j in range(k):
      closest_temp = 0
      for i in range(len(point_list)):
        if i in closest_list:
          continue
        if la.norm(point - point_list[i]) < la.norm(point - point_list[closest_temp]):
          closest_temp = i
        #print(i, closest_temp)
      closest_list.append(closest_temp)

    return closest_list

def dist_sum(point, point_list, closest_list):
    sum = 0
    for i in closest_list:
      sum = sum + la.norm(point - point_list[i])**2

    return sum

def norm_1(K, D_base):
    N = D_base.shape[0]

    sum = 0
    for i in range(N):
        for j in range(N):
            sum += D_base[i][j] - d(i, j, K)

    return sum


def boundary_SDP(base_pose, epsilon, tolerance, max_k, c, sfxn, linear_E_atr_dict, spl, AtomID_to_coords_dict):
    coords = pose_coords_as_rows(base_pose)
    dic = coords_to_AtomID_dict(base_pose)

    D_base = D(base_pose)
    K_base = K_of(D_base)
    N = D_base.shape[0]

    K = cp.Variable((N,N), symmetric=True)
    constraints = [K >> 0]

    for i in range(N):
        constraints += [K[i, i] <= K_base[i, i] + epsilon]
        constraints += [K[i, i] >= K_base[i, i] - epsilon]

        for k in range(2, max_k):
            closest = closest_points(k, coords[i], coords)[1:]
            closest_sum = 0

            for m in closest:
                closest_sum += closest_sum + d(i, m, K)

            constraints += [closest_sum >= dist_sum(coords[i], coords, closest)]

        for j in range(i, N):
            # Ensure pairwise distances deviate at most epsilon elementwise from native configuration
            constraints += [d(i, j, K) <= D_base[i, j]]
            constraints += [d(i, j, K) >= D_base[i, j] - epsilon]

            # Ensure pairwise distances stay above tolerance
            if i != j:
                constraints += [d(i, j, K) >= tolerance]

            # Ensure pairwise distances between bonded atoms remain constant
            id_i = dic[stringify(coords[i])]
            id_j = dic[stringify(coords[j])]
            if base_pose.conformation().is_bonded(id_i, id_j):
                constraints += [d(i, j, K) == D_base[i, j]]

    prob = cp.Problem(cp.Minimize(1e0*(linear_E_atr(K, base_pose, linear_E_atr_dict, spl, AtomID_to_coords_dict) - sfxn(base_pose) - c * norm_1(K, D_base))), constraints)
    prob.solve(verbose = True, solver = "MOSEK", mosek_params = {'MSK_IPAR_INTPNT_SOLVE_FORM':   'MSK_SOLVE_DUAL' })
    return (prob.value, K.value)

In [15]:
lockr_subhelix = slice_pose(lockr, 272, 274)
dump_pdb(lockr_subhelix, 'lockr_subhelix.pdb')
sfxn = ScoreFunction()
sfxn.set_weight(fa_atr, 1.0)

In [16]:
new_min_pose = Pose()
new_min_pose.detached_copy(lockr_subhelix)

relax = rosetta.protocols.relax.ClassicRelax()
relax.set_scorefxn(sfxn)
relax.apply(new_min_pose)
dump_pdb(new_min_pose, 'new_min_pose.pdb')

protocols.relax.ClassicRelax: Setting up default relax setting
protocols.relax.ClassicRelax:
protocols.relax.ClassicRelax:
protocols.relax.ClassicRelax: ===================================================================
protocols.relax.ClassicRelax:    Stage 1
protocols.relax.ClassicRelax:    Ramping repulsives with 8 outer cycles and 1 inner cycles
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 39 rotamers at 3 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.scoring.ScoreFunctionFactory: SCOREFUNCTION: ref2015
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 39 rotamers at 3 positions.
core.pack.interaction_graph.interaction_graph_factory: Instantiating DensePDInteractionGraph
core.pack.task: Packer task: initialize from command line()
core.pack.pack_rotamers: built 39 rotamers at 3 positions.
core.pack.interaction_graph.interaction_graph_

True

In [17]:
const_dict = constant_dictionary(new_min_pose, sfxn)
AtomID_to_coords_dict = AtomID_idx_to_coords_idx_dict(new_min_pose)

<ipython-input-14-7030ad47397f>:8: RuntimeWarning: invalid value encountered in divide
  dir = pos_1 + dist * (pos_2 - pos_1) / la.norm(pos_2 - pos_1)


In [18]:
linear_E_atr_dict, spl = generate_linear_E_atr_dict(new_min_pose, const_dict, AtomID_to_coords_dict)

In [19]:
# value, K = boundary_SDP(new_min_pose, 0.01, 0.6, 10, 0, sfxn, linear_E_atr_dict, spl, AtomID_to_coords_dict)

In [42]:
def jac_E_atr_wrt_d(base_pose, const_dict, spl, AtomID_to_coords_dict):
    D_base = D(base_pose)
    out = []

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    epsilon, sigma = const_dict[((m, j), (n, k))]
                    base_dist = np.sqrt(D_base[AtomID_to_coords_dict[(m, j)]][AtomID_to_coords_dict[(n, k)]])
                    term = 0

                    if base_dist <= sigma:
                        term = 0
                    elif base_dist <= 4.5:
                        term = 6 * epsilon * (sigma ** 6 / base_dist ** 8 - sigma ** 12 / base_dist ** 14)
                    elif base_dist <= 6:
                        coeffs = derive_f_poly(lockr_subhelix, m, j, n, k, sfxn)
                        term = coeffs[1] / (2 * base_dist) + coeffs[2] + 3 * coeffs[3] * base_dist / 2
                    else:
                        term = 0

                    if np.abs(m - n) == 1:
                        out.append(w(m, j, n, k, spl) * term)
                    else:
                        out.append(term)

    return np.array(out)

def jac_d_wrt_alpha(base_pose, delta, AtomID_to_coords_dict):
    D_base = D(base_pose)
    M = num_angles(base_pose)
    out = []
    D_plus = []
    D_minus = []

    for i in range(M):
        noise_plus = np.zeros(M)
        noise_plus[i] = delta
        pose_plus = perturb_pose(base_pose, noise_plus)
        D_plus.append(D(pose_plus))

        noise_minus = np.zeros(M)
        noise_minus[i] = -delta
        pose_minus = perturb_pose(base_pose, noise_minus)
        D_minus.append(D(pose_minus))

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    row = []
                    for l in range(M):
                        d_plus = D_plus[l][AtomID_to_coords_dict[(m, j)]][AtomID_to_coords_dict[(n, k)]]
                        d_minus = D_minus[l][AtomID_to_coords_dict[(m, j)]][AtomID_to_coords_dict[(n, k)]]
                        finite_diff = (d_plus - d_minus) / (2 * delta)
                        row.append(finite_diff)
                    out.append(row)

    return np.array(out)

def derivative_linear_program(base_pose, sfxn, epsilon, delta, tolerance, const_dict, spl, AtomID_to_coords_dict):
    M = num_angles(base_pose)
    alpha = cp.Variable(M)
    D_base = D(base_pose)
    jac_d = jac_d_wrt_alpha(base_pose, delta, AtomID_to_coords_dict)

    constraints = []
    for i in range(M):
        constraints += [alpha[i] >= -epsilon]
        constraints += [alpha[i] <= epsilon]

    i = 0
    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    p = AtomID_to_coords_dict[(m, j)]
                    q = AtomID_to_coords_dict[(n, k)]
                    if D_base[p, q] <= tolerance:
                        constraints += [jac_d[i] @ alpha >= 0]
                    i += 1

    prob = cp.Problem(cp.Minimize(1e6 * jac_E_atr_wrt_d(base_pose, const_dict, spl, AtomID_to_coords_dict) @ jac_d @ alpha), constraints)
    prob.solve()
    # print(prob.get_problem_data(cp.MOSEK))
    return prob.value, alpha.value

def matrix_to_latex(matrix):
    """Converts a NumPy matrix to LaTeX format."""
    matrix[np.abs(matrix) < 1e-10] = 0

    latex_str = "\\begin{pmatrix}"
    for row in matrix:
        latex_str += " & ".join(map(str, row)) + " \\\\"
    latex_str += "\\end{pmatrix}"

    return latex_str

def vector_to_latex(vector):
    """Converts a NumPy vector to LaTeX format."""
    vector[np.abs(vector) < 1e-10] = 0

    latex_str = "\\begin{pmatrix}"
    latex_str += " & ".join(map(str, vector)) + " \\\\"
    latex_str += "\\end{pmatrix}"

    return latex_str

def directional_derivative_linear_program(base_pose, sfxn, epsilon, delta, tolerance, kappa, const_dict, spl, AtomID_to_coords_dict):
    M = num_angles(base_pose)
    alpha = cp.Variable(M)
    D_base = D(base_pose)
    jac_d = jac_d_wrt_alpha(base_pose, delta, AtomID_to_coords_dict)

    constraints = []
    for i in range(M):
        constraints += [alpha[i] >= -epsilon]
        constraints += [alpha[i] <= epsilon]

    constraints += [alpha[1] >= kappa]

    print(matrix_to_latex(jac_d))
    print(jac_E_atr_wrt_d(base_pose, const_dict, spl, AtomID_to_coords_dict) @ jac_d)

    i = 0
    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    p = AtomID_to_coords_dict[(m, j)]
                    q = AtomID_to_coords_dict[(n, k)]
                    if D_base[p, q] <= tolerance:
                        print((m, j), (n, k))
                        print(vector_to_latex(jac_d[i]))
                        constraints += [jac_d[i] @ alpha >= 0]
                    i += 1

    prob = cp.Problem(cp.Minimize(1e6 * jac_E_atr_wrt_d(base_pose, const_dict, spl, AtomID_to_coords_dict) @ jac_d @ alpha), constraints)
    prob.solve()
    return prob.value, alpha.value

def LP_gradient_descent(base_pose, sfxn, epsilon, step_size, delta, tolerance, stop_tol, const_dict, spl, AtomID_to_coords_dict):
     value, alpha = derivative_linear_program(base_pose, sfxn, epsilon, delta, tolerance, const_dict, spl, AtomID_to_coords_dict)
     pose = perturb_pose(base_pose, alpha)

     while step_size > stop_tol:
        value, alpha = derivative_linear_program(pose, sfxn, epsilon, delta, tolerance, const_dict, spl, AtomID_idx_to_coords_idx_dict(pose))
        new_pose = perturb_pose(pose, step_size * alpha)

        while sfxn(new_pose) > sfxn(pose):
            step_size = step_size / 2
            value, alpha = derivative_linear_program(pose, sfxn, epsilon, delta, tolerance, const_dict, spl, AtomID_idx_to_coords_idx_dict(pose))
            new_pose = perturb_pose(pose, step_size * alpha)

        pose = new_pose
        print(value, step_size, alpha, sfxn(pose))
        # step_size = 1

     return pose

In [21]:
def get_angles(pose):
    angles = []

    for i in range(1, pose.total_residue() + 1):
        angles.append(pose.phi(i))
        angles.append(pose.psi(i))

    for i in range(1, pose.total_residue() + 1):
        for j in range(1, num_chis(pose, i) + 1):
            angles.append(pose.chi(j, i))

    return angles

def load_angles(pose, angles):
    for i in range(1, pose.total_residue() + 1):
        pose.set_phi(i, angles[2 * i - 2])
        pose.set_psi(i, angles[2 * i - 1])

    k = 2 * pose.total_residue()
    for i in range(1, pose.total_residue() + 1):
        for j in range(1, num_chis(pose, i) + 1):
            pose.set_chi(j, i, angles[k])
            k += 1

In [30]:
lp_pose = LP_gradient_descent(new_min_pose, sfxn, 1, 1, 0.01, 7, 1e-13, const_dict, spl, AtomID_to_coords_dict)

-29702.28729308844 1 [-1.  0.  1. -0.  1. -1. -1.  1.] -2.583556159216574
-30007.56103680825 1 [-1.  0.  1. -0.  1. -1. -1. -1.] -2.613678304427493
-21360.889410161646 1 [-1.          0.          1.         -0.75067339  1.         -1.
 -1.          1.        ] -2.634992280589854
-21322.63306913434 1 [-1.          0.          1.         -0.74533762  1.         -1.
 -1.          1.        ] -2.656241647044931
-21229.982629068832 1 [-1.          0.          1.         -0.74018494  1.         -1.
 -1.          1.        ] -2.6773696778631377
-21078.28861523736 1 [-1.          0.          1.         -0.73519849  1.         -1.
 -1.          1.        ] -2.698317132285969


KeyboardInterrupt: 

In [31]:
load_angles(lp_pose, [-43.02116800146366,
 -49.62399735165339,
 -43.51265005982482,
 -33.73819259111368,
 -43.92853497337617,
 -43.02116800146366,
 22.497737831746115,
 -6.979587393612171])

In [ ]:
print(sfxn(lp_pose), sfxn(new_min_pose), sfxn(lockr_subhelix))
lp_pose.dump_pdb('lp_pose_265_273.pdb')

-40.45123845812341 -34.990948201580174 -29.232744357585474


True

In [ ]:
get_angles(lp_pose)

[-101.23635295778922,
 -47.24211883222437,
 -46.78572733193324,
 -39.158934795228895,
 -52.401074809019796,
 -45.942257674818904,
 -51.111115437635505,
 -52.77159887467796,
 -63.982992775953036,
 -41.548161184366016,
 -62.46081477972147,
 -45.826184153545945,
 -60.474554955634346,
 -46.01896227720454,
 -56.19753923458908,
 -40.112694982675926,
 -63.29619105897941,
 -101.23635295778922,
 -184.93616230184014,
 61.47058225438347,
 173.26101964783768,
 175.1181494148092,
 -198.2144138150944,
 36.025434938417916,
 -60.490048253629645,
 39.44577938358567,
 -164.27717704543318,
 -67.29339194121991,
 -70.12006251775728,
 -166.6487870586644,
 -68.55804368042023,
 69.81424484181609,
 32.21603197038271]

In [ ]:
derivative_linear_program(lp_pose, sfxn, 1, 0.01, 7, const_dict, spl, AtomID_idx_to_coords_idx_dict(lp_pose))

(-0.0004605279986969626,
 array([-1.00000000e+00,  1.67938124e-15, -6.18683368e-17, -4.56766197e-17,
         2.37354364e-16, -6.45745268e-17, -6.63846141e-16,  3.80527400e-17,
         9.88467852e-16, -1.02733932e-16, -6.83035592e-16,  1.86814912e-16,
         9.83839030e-16,  3.90661433e-16, -1.36447824e-15, -2.26446468e-16,
         2.60904618e-16, -1.00000000e+00, -1.88293201e-15,  4.36230446e-15,
        -1.52933489e-15, -2.84026889e-15,  3.68217981e-01, -3.45093214e-01,
        -3.96887806e-16, -9.99999994e-01, -1.40435658e-16,  8.56586329e-16,
        -3.78010916e-16,  1.92214166e-15, -1.18633790e-15, -4.40675903e-18,
         1.25417079e-15]))

In [43]:
# 272, 274
# 0, 4, 5 (?)

# 273, 278
# 0, 11, 19, 20

directional_derivative_linear_program(lp_pose, sfxn, 1, 0.01, 7, 1e-5, const_dict, spl, AtomID_idx_to_coords_idx_dict(lp_pose))

\begin{pmatrix}0.0 & -0.043032963996036955 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.04549325765221823 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.02099467507683528 & 0.03262696990464775 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.03380644426584922 & 0.05655704024754016 & 0.06587053729347758 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.08653194139647269 & -0.053767426283712894 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.07078853747048974 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.06479720993155524 & 0.013686192584039247 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.08765633591494293 & -0.05309465294267568 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.1291420399002874 & -0.07330558111640073 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & -0.06818571836983978 & -0.06816180820852225 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.0 & -0.03595640575735359 & 0.0 & 0.0 & 0.0 & 0.0 & 0.0 \\0.0 & 0.0 & -0.026518570201261582 & 0.0989

(-0.3554061967051908,
 array([-1.e+00,  1.e-05, -0.e+00, -0.e+00, -0.e+00, -1.e+00, -1.e+00,
        -1.e+00]))

In [ ]:
lp_pose_from_file = pose_from_file('lp_pose.pdb')

core.import_pose.import_pose: File 'lp_pose.pdb' automatically determined to be of type PDB
core.conformation.Conformation: [ WARNING ] missing heavyatom:  OXT on residue SER:CtermProteinFull 3


In [ ]:
sfxn(lp_pose_from_file)

-3.4708302387219163

In [ ]:
pose_coords_as_rows(lp_pose_from_file)

array([[ 9.627     , 14.47      ,  8.747     ],
       [10.127     , 13.362     ,  7.94      ],
       [ 9.209     , 13.086     ,  6.757     ],
       [ 9.67      , 12.952     ,  5.618     ],
       [10.275     , 12.109     ,  8.804     ],
       [10.24692257, 14.62980122,  9.51521844],
       [ 9.57412635, 15.29525115,  8.18471456],
       [ 8.71678246, 14.24540927,  9.09494114],
       [11.003     , 13.593     ,  7.593     ],
       [10.608     , 11.383     ,  8.253     ],
       [10.901     , 12.293     ,  9.522     ],
       [ 9.408     , 11.875     ,  9.171     ],
       [ 7.901     , 12.983     ,  7.011     ],
       [ 6.957     , 12.735     ,  5.927     ],
       [ 7.285     , 13.591     ,  4.71      ],
       [ 7.101     , 13.159     ,  3.566     ],
       [ 5.53      , 13.005     ,  6.403     ],
       [ 7.544     , 13.053     ,  7.79      ],
       [ 7.015     , 11.803     ,  5.663     ],
       [ 4.916     , 12.835     ,  5.671     ],
       [ 5.329     , 12.416     ,  7.147

In [ ]:
pose_coords_as_rows(lp_pose)

array([[ 9.627     , 14.47      ,  8.747     ],
       [10.127     , 13.362     ,  7.94      ],
       [ 9.209     , 13.086     ,  6.757     ],
       [ 9.67031846, 12.95244447,  5.61800167],
       [10.275     , 12.109     ,  8.804     ],
       [ 9.393     , 14.253     ,  9.545     ],
       [11.003     , 13.593     ,  7.593     ],
       [10.608     , 11.383     ,  8.253     ],
       [10.901     , 12.293     ,  9.522     ],
       [ 9.408     , 11.875     ,  9.171     ],
       [ 7.90107443, 12.98335937,  7.01120582],
       [ 6.95700608, 12.73490419,  5.92659761],
       [ 7.28506754, 13.59146099,  4.70970068],
       [ 7.1005726 , 13.15885954,  3.56591984],
       [ 5.53000435, 13.00485912,  6.40298756],
       [ 7.54351278, 13.05319553,  7.79016127],
       [ 7.01487991, 11.8031945 ,  5.66323915],
       [ 4.91619984, 12.83546694,  5.67127429],
       [ 5.32943322, 12.41558612,  7.14704774],
       [ 5.46109453, 13.9303433 ,  6.68468859],
       [ 7.77764291, 14.81029826,  4.939

In [ ]:
load_angles(new_min_pose, get_angles(lp_pose))

In [ ]:
sfxn(new_min_pose)

-3.037018095889514

In [ ]:
LP_gradient_descent(lp_pose, sfxn, 1, 1, 0.01, 7, -1.1176515073429982e-05, const_dict, spl, AtomID_idx_to_coords_idx_dict(lp_pose))

-294.7657535419361 1 [-1.  0. -0. -0. -0. -1.  1.  1.] -3.033593345742924
-1.2105573934384495e-05 5.960464477539063e-08 [-1.  0. -0. -0. -0. -1. -1. -1.] -3.0335933457429243
-1.9799587060553847e-05 5.960464477539063e-08 [-1.  0. -0. -0. -0. -1.  1. -1.] -3.0335933457429243
-9.945833349789268e-06 1.4210854715202004e-14 [-1.  0. -0. -0. -0. -1.  1. -1.] -3.0335933457429243


In [ ]:
D_matrix = D(new_min_pose)
non_zero_values = D_matrix[D_matrix > 0]
min_non_zero = np.min(non_zero_values)
print(np.sort(non_zero_values))


[ 0.738649    0.738649    0.739499    0.739499    0.940625    0.940625
  0.94079     0.94079     0.940854    0.940854    0.941097    0.941097
  0.941134    0.941134    0.941146    0.941146    0.941256    0.941256
  0.941566    0.941566    1.527969    1.527969    1.529417    1.529417
  1.785825    1.785825    2.128913    2.128913    2.12937     2.12937
  2.318389    2.318389    2.322152    2.322152    2.336157    2.336157
  2.338409    2.338409    2.522673    2.522673    2.52431     2.52431
  2.524788    2.524788    2.524969    2.524969    2.5256      2.5256
  2.526974    2.526974    3.842346    3.842346    3.908662    3.908662
  3.918169    3.918169    3.994221    3.994221    3.995144    3.995144
  4.174381    4.174381    4.180134    4.180134    4.196081    4.196081
  4.198761    4.198761    4.240193    4.240193    4.240456    4.240456
  4.241528    4.241528    4.243491    4.243491    4.244561    4.244561
  4.245771    4.245771    5.072198    5.072198    5.417442    5.417442
  5.421445

In [ ]:
sfxn(new_min_pose)

-0.605795992564227

In [ ]:
sfxn(perturb_pose(new_min_pose, [-0.001, -0.001,  0.001, -0.001]))

-0.606146655055105

In [ ]:
jac_E_atr_wrt_d(new_min_pose, const_dict, spl, AtomID_to_coords_dict) @ jac_d_wrt_alpha(new_min_pose, 0.01, AtomID_to_coords_dict)

array([ 0.        ,  0.00155539, -0.00195251,  0.        ])

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.01, AtomID_to_coords_dict)

array([[ 0.00000000e+00, -4.37496967e-02,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -4.62707004e-02,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  1.67051283e-02,  2.07774710e-02,
         0.00000000e+00],
       [ 0.00000000e+00,  3.74072695e-02,  4.74519059e-02,
         0.00000000e+00],
       [ 0.00000000e+00, -1.00508440e-01, -5.41092821e-02,
         0.00000000e+00],
       [ 0.00000000e+00, -7.19569272e-02,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00, -5.44973560e-02,  2.18943446e-02,
         0.00000000e+00],
       [ 0.00000000e+00, -1.01510502e-01, -5.34345433e-02,
         0.00000000e+00],
       [ 0.00000000e+00, -1.40271665e-01, -6.57706607e-02,
         0.00000000e+00],
       [ 0.00000000e+00, -9.32959773e-02, -7.65936319e-02,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
         0.00000000e+00],
       [ 0.00000000e+00,  0.00000000e+00,  0.00000000e+00,
      

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.001, AtomID_to_coords_dict)[0, 1]

-0.04374969692655384

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.0001, AtomID_to_coords_dict)[0, 1]

-0.04374969689635577

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.00001, AtomID_to_coords_dict)[0, 1]

-0.04374969719833643

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.000001, AtomID_to_coords_dict)[0, 1]

-0.043749697198336435

In [ ]:
jac_d_wrt_alpha(new_min_pose, 0.0000001, AtomID_to_coords_dict)[0, 1]

-0.04374968831655224

In [ ]:
AtomID_to_coords_dict

{(1, 1): 0,
 (1, 2): 1,
 (1, 3): 2,
 (1, 4): 3,
 (1, 5): 4,
 (1, 6): 5,
 (1, 7): 6,
 (1, 8): 7,
 (1, 9): 8,
 (1, 10): 9,
 (2, 1): 10,
 (2, 2): 11,
 (2, 3): 12,
 (2, 4): 13,
 (2, 5): 14,
 (2, 6): 15,
 (2, 7): 16,
 (2, 8): 17,
 (2, 9): 18,
 (2, 10): 19}

In [ ]:
for k in range(2, 17):
    value, K = boundary_SDP(new_min_pose, 0.01, 0.6, k, 0, sfxn, linear_E_atr_dict, spl, AtomID_to_coords_dict)
    print(k, value, la.svd(K)[1], np.sum(la.svd(K)[1]))

2 -0.7084364294056567 [1.02533284e+02 2.40283518e+01 1.28701639e+01 5.79949393e-02
 2.05157840e-03 5.38996244e-04 1.50734970e-08 6.80816829e-09
 6.72921013e-09 6.72238171e-09 6.54129514e-09 6.42650701e-09
 6.27349545e-09 5.79730436e-09 5.58988307e-09 5.47460988e-09
 5.04160133e-09 4.42154937e-09 3.08479562e-09 2.77135000e-09] 139.49238523747155
3 -0.7084364294056567 [1.02533284e+02 2.40283518e+01 1.28701639e+01 5.79949393e-02
 2.05157840e-03 5.38996244e-04 1.50734970e-08 6.80816829e-09
 6.72921013e-09 6.72238171e-09 6.54129514e-09 6.42650701e-09
 6.27349545e-09 5.79730436e-09 5.58988307e-09 5.47460988e-09
 5.04160133e-09 4.42154937e-09 3.08479562e-09 2.77135000e-09] 139.49238523747155
4 -0.7084349995056805 [1.02533284e+02 2.40283518e+01 1.28701639e+01 5.53613453e-02
 2.05180447e-03 5.39074608e-04 1.12311606e-08 6.52022665e-09
 6.45385369e-09 6.44217450e-09 6.34311966e-09 6.16468499e-09
 5.76799176e-09 5.44888050e-09 5.15776635e-09 4.28833163e-09
 3.80008179e-09 3.78077235e-09 2.1237214

/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #12 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #85 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #156 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #223 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnin

12 -0.7082310815283988 [1.02533343e+02 2.40283064e+01 1.28701267e+01 6.09787212e-02
 2.02560682e-03 5.51851251e-04 3.66310822e-06 8.59417018e-07
 6.23953025e-07 4.91248101e-07 3.53992394e-07 3.14700610e-07
 2.16513858e-07 1.86041437e-07 1.37249400e-07 9.60640654e-08
 7.26325342e-08 3.04890759e-08 1.59149391e-08 1.08858110e-08] 139.4953397127215


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #13 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #87 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #159 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #227 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnin

13 -0.7078548762268211 [1.02533424e+02 2.40282432e+01 1.28701379e+01 5.75396384e-02
 1.96455849e-03 5.67513855e-04 1.68821561e-05 4.70801628e-06
 3.37389878e-06 1.59389256e-06 1.26442740e-06 6.66043259e-07
 4.85056923e-07 3.44662589e-07 2.25999239e-07 1.38083486e-07
 8.09194651e-08 6.76503178e-08 5.11352577e-08 2.00135310e-08] 139.49190672994408


...

14 -0.7080220331400211 [1.02533353e+02 2.40283012e+01 1.28701569e+01 5.75905090e-02
 1.98277560e-03 5.61747684e-04 6.98518488e-06 2.35840827e-06
 1.52173034e-06 8.50632993e-07 7.72038406e-07 6.98461040e-07
 4.24151212e-07 3.77413078e-07 2.99105007e-07 1.90853276e-07
 1.72055600e-07 7.37264785e-08 5.42366951e-08 3.29881758e-08] 139.4919613354106


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #15 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #90 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #91 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #163 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warning

15 -0.7071438388972506 [1.02533464e+02 2.40282219e+01 1.28701755e+01 5.84253738e-02
 1.87545933e-03 5.93042610e-04 2.50299855e-05 7.12056750e-06
 4.35134686e-06 3.54728601e-06 2.65844181e-06 1.72419378e-06
 1.47761774e-06 1.05538045e-06 9.58092112e-07 5.75328527e-07
 5.60328852e-07 2.28083726e-07 1.64986364e-07 1.46444851e-07] 139.49280538675453


/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #16 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #92 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #93 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warnings.warn(f"Constraint #{i} contains too many subexpressions. "
/usr/local/lib/python3.10/dist-packages/cvxpy/problems/problem.py:163: UserWarning: Constraint #166 contains too many subexpressions. Consider vectorizing your CVXPY code to speed up compilation.
  warning

16 -0.7077149805130301 [1.02533407e+02 2.40282801e+01 1.28701032e+01 5.72547237e-02
 1.97885952e-03 5.62214723e-04 1.16192641e-05 3.31637157e-06
 2.45986342e-06 1.56102707e-06 1.52919034e-06 1.23359679e-06
 8.76983365e-07 6.55785452e-07 4.03407017e-07 3.16511399e-07
 2.22499448e-07 1.37326506e-07 1.11803516e-07 8.64027508e-08] 139.4916100958092


In [ ]:
# prompt: rank of matrix K
0.7425
print(la.matrix_rank(K))


In [ ]:
dump_pdb(new_min_pose, 'new_min_pose.pdb')

In [ ]:
def d_ij_helper(i, j, N):
    A = np.zeros((N, N))
    A[i, i] = 1
    A[i, j] = -1
    A[j, i] = -1
    A[j, j] = 1
    return A

# def min_slope(linear_E_atr_dict):


def linear_E_atr_x(x, base_pose, min_slope, tolerance, linear_E_atr_dict, spl, AtomID_to_coords_dict):
    sum = 0
    D_base = D(base_pose)
    N = D_base.shape[0]
    x_base_T, base_sv, x_base = la.svd(K(D_base))
    x_base_flat = x_base[0:3].flatten()

    for m in range(1, base_pose.total_residue() + 1):
        for n in range(m + 1, base_pose.total_residue() + 1):
            for j in range(1, base_pose.residue(m).natoms() + 1):
                for k in range(1, base_pose.residue(n).natoms() + 1):
                    constants = linear_E_atr_dict[((m, j), (n, k))]
                    # dist_squared = cp.quad_form(x, np.kron(np.identity(3), d_ij_helper(AtomID_to_coords_dict[(m, j)], AtomID_to_coords_dict[(n, k)], N)), True)
                    dist_squared = x_base_flat.T @ np.kron(np.identity(3), d_ij_helper(AtomID_to_coords_dict[(m, j)], AtomID_to_coords_dict[(n, k)], N)) @ x
                    term = 0

                    if base_pose.conformation().is_bonded(AtomID(j, m), AtomID(k, n)):
                        if type(constants) == tuple:
                            if np.abs(m - n) == 1:
                                slope, intercept, base_dist = constants
                                term = cp.maximum(w(m, j, n, k, spl) * (intercept + slope * (dist_squared - base_dist ** 2)), N * min_slope * (base_dist ** 2 - dist_squared) + w(m, j, n, k, spl) * intercept)
                            else:
                                slope, intercept, base_dist = constants
                                term = cp.maximum(intercept + slope * (dist_squared - base_dist ** 2), N * min_slope * (base_dist ** 2 - dist_squared) + intercept)
                        else:
                            if np.abs(m - n) == 1:
                                term = cp.maximum(w(m, j, n, k, spl) * constants, N * min_slope * (base_dist ** 2 - dist_squared) + w(m, j, n, k, spl) * constants)
                            else:
                                term = cp.maximum(constants, N * min_slope * (base_dist ** 2 - dist_squared) + constants)
                    else:
                        if type(constants) == tuple:
                            if np.abs(m - n) == 1:
                                slope, intercept, base_dist = constants
                                term = cp.maximum(w(m, j, n, k, spl) * (intercept + slope * (dist_squared - base_dist ** 2)), N * min_slope * (tolerance - dist_squared) + w(m, j, n, k, spl) * (intercept + slope * (tolerance - base_dist ** 2)))
                            else:
                                slope, intercept, base_dist = constants
                                term = cp.maximum(intercept + slope * (dist_squared - base_dist ** 2), N * min_slope * (tolerance - dist_squared) + intercept + slope * (tolerance - base_dist ** 2))
                        else:
                            if np.abs(m - n) == 1:
                                term = cp.maximum(w(m, j, n, k, spl) * constants, N * min_slope * (tolerance - dist_squared) + w(m, j, n, k, spl) * constants)
                            else:
                                term = cp.maximum(constants, N * min_slope * (tolerance - dist_squared) + constants)

                    sum += term
    return sum

def funnel_qcqp(base_pose, norms, epsilon, tolerance, sfxn, linear_E_atr_dict, spl, AtomID_to_coords_dict):
    coords = pose_coords_as_rows(base_pose)
    dic = coords_to_AtomID_dict(base_pose)
    D_base = D(base_pose)
    N = D_base.shape[0]

    # x = [x_1, x_2, x_3]^T where x_i are size (N, 1)
    x = cp.Variable((3 * N, 1))

    x_base_T, base_sv, x_base = la.svd(K(D_base))

    constraints = []

    # Given norms = [c_1, c_2, c_3], ensure that x_i^T x_i = c_i
    constraints += [cp.quad_form(x, np.kron([[1, 0, 0],
                                             [0, 0, 0],
                                             [0, 0, 0]], np.identity(N)), True) <= norms[0]]

    constraints += [x_base_T[0] @ x[0 : N] >= base_sv[0] - epsilon]

    constraints += [cp.quad_form(x, np.kron([[0, 0, 0],
                                             [0, 1, 0],
                                             [0, 0, 0]], np.identity(N)), True) <= norms[1]]

    constraints += [x_base_T[1] @ x[N : 2 * N] >= base_sv[1] - epsilon]

    constraints += [cp.quad_form(x, np.kron([[0, 0, 0],
                                             [0, 0, 0],
                                             [0, 0, 1]], np.identity(N)), True) <= norms[2]]

    constraints += [x_base_T[2] @ x[2 * N : 3 * N] >= base_sv[2] - epsilon]

    # Ensure x_1^T x_2 = 0
    constraints += [cp.quad_form(x, np.kron([[1, 1, 0],
                                             [1, 1, 0],
                                             [0, 0, 0]], np.identity(N)), True) - norms[0] - norms[1] <= 0]

    constraints += [cp.quad_form(x, np.kron([[1, -1, 0],
                                             [-1, 1, 0],
                                             [0, 0, 0]], np.identity(N)), True) - norms[0] - norms[1] <= 0]

    # Ensure x_1^T x_3 = 0
    constraints += [cp.quad_form(x, np.kron([[1, 0, 1],
                                             [0, 0, 0],
                                             [1, 0, 1]], np.identity(N)), True) - norms[0] - norms[2] <= 0]

    constraints += [cp.quad_form(x, np.kron([[1, 0, -1],
                                             [0, 0, 0],
                                             [-1, 0, 1]], np.identity(N)), True) - norms[0] - norms[2] <= 0]

    # Ensure x_2^T x_3 = 0
    constraints += [cp.quad_form(x, np.kron([[0, 0, 0],
                                             [0, 1, 1],
                                             [0, 1, 1]], np.identity(N)), True) - norms[1] - norms[2] <= 0]

    constraints += [cp.quad_form(x, np.kron([[0, 0, 0],
                                             [0, 1, -1],
                                             [0, -1, 1]], np.identity(N)), True) - norms[1] - norms[2] <= 0]

    print(1)

    for i in range(N):
        for j in range(i + 1, N):
            # Ensure pairwise distances devaite at most epsilon elementwise from native configuration
            constraints += [cp.quad_form(x, np.kron(np.identity(3), d_ij_helper(i, j, N)), True) <= D_base[i, j] + epsilon]

            # Ensure pairwise distances stay above tolerance
            # constraints += [cp.quad_form(x, np.kron(np.identity(3), d_ij_helper(i, j, N)), True) >= tolerance]

            # Ensure pairwise distances between bonded atoms remain constant
            id_i = dic[stringify(coords[i])]
            id_j = dic[stringify(coords[j])]
            if base_pose.conformation().is_bonded(id_i, id_j):
                constraints += [cp.quad_form(x, np.kron(np.identity(3), d_ij_helper(i, j, N)), True) <= D_base[i, j]]

    print(2)


    prob = cp.Problem(cp.Minimize(linear_E_atr_x(x, base_pose, 10, tolerance, linear_E_atr_dict, spl, AtomID_to_coords_dict) - sfxn(base_pose)), constraints)
    prob.solve(verbose = True)
    return (prob.value, x.value)

In [ ]:
funnel_qcqp(new_min_pose, [7.87810778e+02 + 0.01, 4.70327268e+02 + 0.01, 2.56280380e+02 + 0.01], 0.01, 0.01, sfxn, linear_E_atr_dict, spl, AtomID_to_coords_dict)

In [ ]:
# prompt: print the singular values of K(D(new_min_pose))

import numpy.linalg as la
K_D = K(D(new_min_pose))

la.svd(K_D)[0][0:3].flatten() @ np.kron(np.identity(3), d_ij_helper(AtomID_to_coords_dict[(1, 1)], AtomID_to_coords_dict[(5, 1)], K_D.shape[0]))

In [ ]:
# prompt: flatten numpy array x = [[1, 1], [0, 0]]

import numpy as np

x = np.array([[1, 1], [0, 0]])
x_flat = x.flatten()
print(x_flat)


In [ ]:
X = cp.Variable((100, 100), symmetric = True)
constraints = [X >> 0]
prob = cp.Problem(cp.Minimize(cp.norm(X, "fro")), constraints)
prob.solve(verbose = True, solver = "MOSEK", mosek_params = {'MSK_DPAR_INTPNT_TOL_STEP_SIZE': 1.0e-6})

                                     CVXPY                                     
                                     v1.3.4                                    
(CVXPY) Jul 08 10:27:21 PM: Your problem has 10000 variables, 1 constraints, and 0 parameters.
(CVXPY) Jul 08 10:27:21 PM: It is compliant with the following grammars: DCP, DQCP
(CVXPY) Jul 08 10:27:21 PM: (If you need to solve this problem multiple times, but with different data, consider using parameters.)
(CVXPY) Jul 08 10:27:21 PM: CVXPY will first compile your problem; then, it will invoke a numerical solver to obtain a solution.
-------------------------------------------------------------------------------
                                  Compilation                                  
-------------------------------------------------------------------------------
(CVXPY) Jul 08 10:27:21 PM: Compiling problem (target solver=MOSEK).
(CVXPY) Jul 08 10:27:21 PM: Reduction chain: Dcp2Cone -> CvxAttr2Constr -> ConeMatrixStuffin

4.014580810248894e-15